<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/finding_noisepollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade

In [ ]:
# mkdir static
import fitz

doc = fitz.open("dragged_130_146.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)


In [ ]:
text=('output3.txt')

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    # Print the contents
    print(contents)

In [ ]:
import re
split_contents = re.split(r'\d+\.', contents)
split_contents

In [ ]:
len(split_contents)

68

In [ ]:
!pip install sentence_transformers
#also works with entire alineas 
#below are some other models that don't work. I get a warning signal that says it can't find the model so it switches to MEAN pooling instead. 
#('all-MiniLM-L6-v2'), one of the most popular ones, works but it is an English model 
#("uer/sbert-base-chinese-nli")#('shibing624/text2vec-base-chinese')#('all-MiniLM-L6-v2') 

from sentence_transformers import SentenceTransformer, util
import torch

#alternative model by same creators: ("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"). This one appears to be more popular. Compare results.
embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# Corpus with example sentences
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['噪音', '水下噪音','聲音']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

#calculate accumulated similarity score per keyword

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

36

In [ ]:
import pandas as pd
#in case we use more or less keywords, we will have to adjust this code slightly
df = pd.DataFrame([[sentence, scores[queries[0]], scores[queries[1]], 
                    scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]] 
                   for sentence, scores in sentence_scores.items()], 
                  columns=['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 
                           'accumulated_score'])
df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
#output
df=df.reset_index(drop=True)
df

,text,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,,tensor(0.4494),tensor(0.2770),tensor(0.4720),tensor(1.1983)
1,開發行為屬點狀開\n發 ， 無 大 面 積 施\n工，環境影響說明\n書中已針對施工及\n營...,tensor(0.3082),tensor(0.3964),tensor(0.2690),tensor(0.9736)
2,2-2「施工階段環境\n監測計畫表」中「水下噪音」之第一\n項監測內容相符，如表,tensor(0.3069),tensor(0.4042),tensor(0.2564),tensor(0.9675)
3,1 \n8-1 \n(二)施工期間於距離打樁\n位置外750公尺處選擇\n合理方位設置4座水...,tensor(0.2620),tensor(0.4287),tensor(0.2600),tensor(0.9507)
4,2 \n8-4 \n8-6~7 \n8-26 \n \n10\n審查意見 \n�覆說明 \n...,tensor(0.2729),tensor(0.3951),tensor(0.2385),tensor(0.9064)
5,1 \n8-5 \n(五)施工前不使用聲音驅\n離裝置(ADD)。 \n遵照辦理，「不使用聲...,tensor(0.2564),tensor(0.2333),tensor(0.3022),tensor(0.7919)
6,5)背景�度\n即已超過空氣品質\n標準外，其餘均可\n符合環境空氣品質\n標準，開發單位已...,tensor(0.2625),tensor(0.2446),tensor(0.2157),tensor(0.7228)
7,1 \n \n,tensor(0.2642),tensor(0.1417),tensor(0.2779),tensor(0.6838)
8,2 \n,tensor(0.2584),tensor(0.1449),tensor(0.2691),tensor(0.6725)
9,1 \n8-4 \n(四)打樁工程應採緩啟動\n(softstart)持續至少30\n分鐘。...,tensor(0.1974),tensor(0.2242),tensor(0.1625),tensor(0.5841)
